<a href="https://colab.research.google.com/github/waltersbrf/Doutorado/blob/main/FAQ_com_Langchain_e_LLMs_OpenAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo do Notebook 

# Introdução

LangChain é um framework para desenvolvimento de aplicações usando Grandes Modelos de Linguagem (Large Language Model - LLMs). Com a combinação  das LLMs com outros ativos é possível gerar aplicações que envolvem gerenciamento de prompt, encadeamento (chain), geração de dados aumentados(data augmentation), designação de sequência de operações, etc.


No caso deste notebook, será apresentado um processo de criação de FAQ (Frequently Asked Questions Em português seria algo como "Perguntas frequentes") sobre um tema. No caso, o tema em questão envolve atores de um sistema de rastreio de satélites.


Para trabalhar num tema com LangChain, será necessário informar alguns ativos para servir de base para o processamento, no caso esse exemplo irá fornecer um conjunto de manuais e artigos sobre o tema. 

Como observação, as respostas devem variar de acordo com o material de suporte fornecido. 

Os esforços destes notebook focam na execução deste conjunto no Google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "seu token aqui - OpenIA"

In [ ]:

pdf_folder_path = '/content/gdrive/MyDrive/data2'

# Instalando dependências 

In [ ]:
!pip install openai

!pip install langchain
!pip install sentence_transformers
!pip install faiss-cpu

!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

!pip install pdf2image
!pip install pytesseract

!pip install PyPDF2

!apt-get install poppler-utils tesseract-ocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 23.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.7/960.7 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 5,024 kB of archives.
After this operation, 17.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/

# Implementando as Operações

In [ ]:
import os 

from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator


loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

In [ ]:
#from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
#from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
index = VectorstoreIndexCreator(
    embedding= OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=2500, chunk_overlap=0)).from_loaders(loaders)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


RateLimitError: ignored

In [ ]:

chain = RetrievalQA.from_chain_type(llm=OpenAI() ,
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")


In [ ]:
chain.run('Os textos falam sobre o que?')

NameError: ignored